In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 23.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

dataset_positive_folder= "/content/drive/path1"
dataset_negative_folder= "/content/drive/path2"

In [ ]:
# code was partially adapted from https://www.analyticsvidhya.com/blog/2021/10/human-pose-estimation-using-machine-learning-in-python/
import mediapipe as mp
import cv2
import time
import numpy as np
import pandas as pd
import os
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks
path = dataset_positive_folder
data = []
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data)

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:

count = 0

for image in os.listdir(path):
        temp = []
        img = cv2.imread(path + "/" + image)
        imageWidth, imageHeight = img.shape[:2]
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(imgRGB)
        if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                for i,j in zip(points,landmarks):
                        temp = temp + [j.x, j.y, j.z, j.visibility]
                data.loc[count] = temp
                count +=1
        else:
            print(image , " has no landmarks")
data.to_csv("dataset_positive.csv") # save the data as a csv file

Open dataset
Add y-label: 1 for positive and 0 for negative pose
shuffle dataset

In [ ]:
import pandas as pd
df = pd.read_csv('dataset_positive.csv', sep=',', index_col=0 )
df.head()

In [ ]:
print(df.columns)

In [ ]:
array1 = np.ones(df.shape[0])
array1

In [ ]:
df.shape

In [ ]:
df['target'] = array1
df.head()

In [ ]:
# save positive dataset with y-label
df.to_csv("dataset_positive_labeled.csv") # save the data as a csv file

In [ ]:
mpPose = mp.solutions.pose
pose = mpPose.Pose()
mpDraw = mp.solutions.drawing_utils # For drawing keypoints
points = mpPose.PoseLandmark # Landmarks
path = dataset_negative_folder
data = []
for p in points:
        x = str(p)[13:]
        data.append(x + "_x")
        data.append(x + "_y")
        data.append(x + "_z")
        data.append(x + "_vis")
data = pd.DataFrame(columns = data) # Empty dataset

In [ ]:
count = 0

for image in os.listdir(path):

        temp = []
        img = cv2.imread(path + "/" + image)
        imageWidth, imageHeight = img.shape[:2]
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(imgRGB)
        if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                for i,j in zip(points,landmarks):
                        temp = temp + [j.x, j.y, j.z, j.visibility]
                data.loc[count] = temp
                count +=1
        else:
          print(image , " has bo landmarks")


In [ ]:
data.shape

In [ ]:
data.to_csv("dataset_negative.csv") # save the data as a csv file

Add y-label: 1 for positive pose and 0 for negative pose.

In [ ]:
df_negative = pd.read_csv('dataset_negative.csv', sep=',', index_col=0 )
df_negative = df_negative.sample(frac=1).reset_index(drop=True)
df_negative.head()



In [ ]:
df2 = df_negative.iloc[:df_negative.shape[0]].copy()

In [ ]:
array0 = np.zeros(df2.shape[0])
array0

In [ ]:
print(df.shape)
print(df2.shape)

(921, 133)
(854, 132)


In [ ]:
df2['target'] = array0
df2.head()

In [ ]:
df2.to_csv("dataset_robot_negative_labeled.csv") # save the data as a csv file

Combine two datasets to one

In [ ]:
frames = [df.iloc[:df2.shape[0]], df2]

dataset_full = pd.concat(frames)
dataset_full.head()

In [ ]:
dataset_full.shape

In [ ]:
dataset_full.to_csv("dataset_full_labeled.csv") # save the data as a csv file

In [ ]:
#dataset_full.shape
dataset_shuffled= dataset_full.sample(frac=1).reset_index(drop=True)
dataset_shuffled.head()

In [ ]:
dataset_shuffled.to_csv("dataset_full_shuffled_labeled.csv") # save the data as a csv file